<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>


<h1>Data Wrangling with Regular Expressions</h1>

Estimated time needed: **40** minutes


## Lab Overview:

In the previous data collection labs, you collected some raw datasets from several different sources. In this lab, you need to perform data wrangling tasks in order to improve data quality.


You will again use regular expressions, along with the `stringr` package (part of `tidyverse`), to clean up the bike-sharing systems data that you previously web scraped from the wiki page: 

https://en.wikipedia.org/wiki/List_of_bicycle-sharing_systems

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/l2-list-bike-sharing-systems.png" width="800" align="center">
</a>


One typical challenge of web scraping is that data extracted from HTML pages may contain unnecessary or inconsistently fomatted information.  
For example:
 - Textual annotations in numeric fields: `1000 (Updated with 1050)`
 - Attached reference links: `Bike sharing system [123]`
 - Inconsistent data formats: `Yes` and `Y` for the logical value `TRUE` or `2021-04-09` and `Apr 09, 2021` for the same date
 - HTML style tags: `<span style="color:blue">Bike sharing system</span>`
 - Special characters: `&nbsp` for a white space
 
Many more such examples of noise may be encountered in real-world scraped data and most of such text related noises could be handled by regular expressions.


To summarize, you will be using `stringr` (part of `tidyverse`) and regular expressions to perform the following data wrangling tasks:
  - TASK: Standardize column names for all collected datasets
  - TASK: Remove undesired reference links from the scraped bike-sharing systems dataset
  - TASK: Extract only the numeric value from undesired text annotations


Let's begin by importing the libraries you will use for these data wrangling tasks. 

Please note that the require("tidyverse")" command is commented here as the tidyverse package is already pre-installed in this lab environment. However, if you are executing this lab local R-Studio on your system then install this package first and then load the package.


In [1]:
# Check whether you need to install the `tidyverse` library
# require("tidyverse")
install.packages("tidyverse")
library(tidyverse)
library(readr)
library(stringr)

Installing package into ‘/opt/conda/envs/R-RT24.1/lib/R/extra-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## TASK: Standardize column names for all collected datasets 


In the previous data collection labs, you collected four datasets in csv format:
- `raw_bike_sharing_systems.csv`:  A list of active bike-sharing systems across the world
- `raw_cities_weather_forecast.csv`: 5-day weather forecasts for a list of cities, from OpenWeather API
- `raw_worldcities.csv`: A list of major cities' info (such as name, latitude and longitude) across the world
- `raw_seoul_bike_sharing.csv`: Weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour, and date information, from Seoul bike-sharing systems


*Optional:* If you had some difficulties finishing the data collection labs, you may download the datasets directly from the following URLs:


In [2]:
# Download raw_bike_sharing_systems.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_bike_sharing_systems.csv"
download.file(url, destfile = "raw_bike_sharing_systems.csv")

# Download raw_cities_weather_forecast.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_cities_weather_forecast.csv"
download.file(url, destfile = "raw_cities_weather_forecast.csv")

# Download raw_worldcities.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_worldcities.csv"
download.file(url, destfile = "raw_worldcities.csv")

# Download raw_seoul_bike_sharing.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
download.file(url, destfile = "raw_seoul_bike_sharing.csv")

print("done")

[1] "done"


To improve dataset readbility by both human and computer systems, we first need to standardize the column names of the datasets above using the following naming convention:
 - Column names need to be UPPERCASE
 - The word separator needs to be an underscore, such as in `COLUMN_NAME`


You can use the following dataset list and the `names()` function to get and set each of their column names, and convert them according to our defined naming convention.


In [3]:
dataset_list <- c('raw_bike_sharing_systems.csv', 'raw_seoul_bike_sharing.csv', 'raw_cities_weather_forecast.csv', 'raw_worldcities.csv')

_TODO_: Write a `for` loop to iterate over the above datasets and convert their column names 


In [4]:
for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name)
    # Standardized its columns:
    
    # Convert all column names to uppercase
    names(dataset) <- toupper(names(dataset))
    # Replace any white space separators by underscores, using the str_replace_all function
    names(dataset) <- str_replace_all(names(dataset), " ", "_")
  
    # Save the dataset 
    write.csv(dataset, dataset_name, row.names=FALSE)
}


Rows: 480 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): COUNTRY, City, Name, SYSTEM, OPERATOR, LAUNCHED, DISCONTINUED, STA...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 8760 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): Date, SEASONS, HOLIDAY, FUNCTIONING_DAY
dbl (10): RENTED_BIKE_COUNT, Hour, TEMPERATURE, HUMIDITY, WIND_SPEED, Visibi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 160 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): city, weather, season
dbl  (8): visibility, temp, temp_min, temp_max, pressure, humidity, wind_spe...
dttm (1): for

_TODO_: Read the resulting datasets back and check whether their column names follow the naming convention


In [5]:
for (dataset_name in dataset_list){
    # Print a summary for each data set to check whether the column names were correctly converted
    # Read the dataset
    dataset <- read_csv(dataset_name)
    print(colnames(dataset))
}

Rows: 480 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): COUNTRY, CITY, NAME, SYSTEM, OPERATOR, LAUNCHED, DISCONTINUED, STA...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 [1] "COUNTRY"         "CITY"            "NAME"            "SYSTEM"         
 [5] "OPERATOR"        "LAUNCHED"        "DISCONTINUED"    "STATIONS"       
 [9] "BICYCLES"        "DAILY_RIDERSHIP"


Rows: 8760 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DATE, SEASONS, HOLIDAY, FUNCTIONING_DAY
dbl (10): RENTED_BIKE_COUNT, HOUR, TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBI...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 [1] "DATE"                  "RENTED_BIKE_COUNT"     "HOUR"                 
 [4] "TEMPERATURE"           "HUMIDITY"              "WIND_SPEED"           
 [7] "VISIBILITY"            "DEW_POINT_TEMPERATURE" "SOLAR_RADIATION"      
[10] "RAINFALL"              "SNOWFALL"              "SEASONS"              
[13] "HOLIDAY"               "FUNCTIONING_DAY"      


Rows: 160 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): CITY, WEATHER, SEASON
dbl  (8): VISIBILITY, TEMP, TEMP_MIN, TEMP_MAX, PRESSURE, HUMIDITY, WIND_SPE...
dttm (1): FORECAST_DATETIME

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 [1] "CITY"              "WEATHER"           "VISIBILITY"       
 [4] "TEMP"              "TEMP_MIN"          "TEMP_MAX"         
 [7] "PRESSURE"          "HUMIDITY"          "WIND_SPEED"       
[10] "WIND_DEG"          "SEASON"            "FORECAST_DATETIME"


Rows: 26569 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): CITY, CITY_ASCII, COUNTRY, ISO2, ISO3, ADMIN_NAME, CAPITAL
dbl (4): LAT, LNG, POPULATION, ID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 [1] "CITY"       "CITY_ASCII" "LAT"        "LNG"        "COUNTRY"   
 [6] "ISO2"       "ISO3"       "ADMIN_NAME" "CAPITAL"    "POPULATION"
[11] "ID"        


## Process the web-scraped bike sharing system dataset 


By now we have standardized all column names. Next, we will focus on cleaning up the values in the web-scraped bike sharing systems dataset.


In [6]:
# First load the dataset
bike_sharing_df <- read_csv("raw_bike_sharing_systems.csv")

Rows: 480 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): COUNTRY, CITY, NAME, SYSTEM, OPERATOR, LAUNCHED, DISCONTINUED, STA...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
# Print its head
head(bike_sharing_df)

COUNTRY,CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Albania,Tirana,Ecovolis,NA,NA,March 2011,NA,8,200,NA
Argentina,Mendoza,Metrobici,NA,NA,2014,NA,2,40,NA
Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,NA,27 November 2016,NA,8,80,NA
Argentina,Buenos Aires,Ecobici,Serttel Brasil,Bike In Baires Consortium.[10],2010,NA,400,4000,21917
Argentina,Rosario,Mi Bici Tu Bici[11],NA,NA,2 December 2015,NA,47,480,NA
Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,NA


Even from the first few rows, you can see there is plenty of undesireable embedded textual content, such as the reference link included in `Melbourne[12]`.


In this project, let's only focus on processing the following revelant columns (feel free to process the other columns for more practice):
 - `COUNTRY`: Country name 
 - `CITY`: City name
 - `SYSTEM`: Bike-sharing system name
 - `BICYCLES`: Total number of bikes in the system


In [8]:
# Select the four columns
sub_bike_sharing_df <- bike_sharing_df %>% select(COUNTRY, CITY, SYSTEM, BICYCLES)

Let's see the types of the selected columns


In [9]:
sub_bike_sharing_df %>% 
    summarize_all(class) %>%
    gather(variable, class)

variable,class
<chr>,<chr>
COUNTRY,character
CITY,character
SYSTEM,character
BICYCLES,character


They are all interpreted as character columns, but we expect the `BICYCLES` column to be of numeric type. Let's see why it wasn't loaded as a numeric column - possibly some entries contain characters. Let's create a simple function called `find_character` to check that.


In [10]:
# grepl searches a string for non-digital characters, and returns TRUE or FALSE
# if it finds any non-digital characters, then the bicyle column is not purely numeric
find_character <- function(strings) grepl("[^0-9]", strings)

Let's try to find any elements in the `Bicycles` column containing non-numeric characters.


In [11]:
sub_bike_sharing_df %>% 
    select(BICYCLES) %>% 
    filter(find_character(BICYCLES)) %>%
    slice(0:10)

BICYCLES
<chr>
4115[22]
310[59]
500[72]
[75]
180[76]
600[77]
[78]
initially 800 (later 2500)
100 (220)


As you can see, many rows have non-numeric characters, such as `32 (including 6 rollers) [162]` and `1000[253]`. This is actually very common for a table scraped from Wiki when no input validation is enforced. 

Later, you will use regular expressions to clean them up.


Next, let's take a look at the other columns, namely `COUNTRY`, `CITY`, and `SYSTEM`, to see if they contain any undesired reference links, such as in `Melbourne[12]`.


In [12]:
# Define a 'reference link' character class, 
# `[A-z0-9]` means at least one character 
# `\\[` and `\\]` means the character is wrapped by [], such as for [12] or [abc]
ref_pattern <- "\\[[A-z0-9]+\\]"
find_reference_pattern <- function(strings) grepl(ref_pattern, strings)

In [13]:
# Check whether the COUNTRY column has any reference links
sub_bike_sharing_df %>% 
    select(COUNTRY) %>% 
    filter(find_reference_pattern(COUNTRY)) %>%
    slice(0:10)

COUNTRY
<chr>


Ok, looks like the `COUNTRY` column is clean. Let's check the `CITY` column.


In [14]:
# Check whether the CITY column has any reference links
sub_bike_sharing_df %>% 
    select(CITY) %>% 
    filter(find_reference_pattern(CITY)) %>%
    slice(0:10)

CITY
<chr>
Melbourne[12]
Brisbane[14][15]
Lower Austria[18]
Namur[19]
Brussels[21]
Salvador[23]
Belo Horizonte[24]
João Pessoa[25]
(Pedro de) Toledo[26]


Hmm, looks like the `CITY` column has some reference links to be removed. Next, let's check the `SYSTEM` column.


In [15]:
# Check whether the System column has any reference links
sub_bike_sharing_df %>% 
    select(SYSTEM) %>% 
    filter(find_reference_pattern(SYSTEM)) %>%
    slice(0:10)

SYSTEM
<chr>
EasyBike[58]
4 Gen.[61]
3 Gen. SmooveKey[113]
3 Gen. Smoove[141][142][143][139]
3 Gen. Smoove[179]
3 Gen. Smoove[181]
3 Gen. Smoove[183]


So the `SYSTEM` column also has some reference links.


After some preliminary investigations, we identified that the `CITY` and `SYSTEM` columns have some undesired reference links, and the `BICYCLES` column has both reference links and some 
textual annotations.

Next, you need to use regular expressions to clean up the unexpected reference links and text annotations in numeric values.


# TASK: Remove undesired reference links using regular expressions


_TODO:_ Write a custom function using `stringr::str_replace_all` to replace all reference links with an empty character for columns `CITY` and `SYSTEM`


In [16]:
# remove reference link
remove_ref <- function(strings) {
    # ref_pattern <- "Define a pattern matching a reference link such as [1]"
    ref_pattern <- "\\[[A-z0-9]+\\]"
  
    # Replace all matched reference links with an empty string (remove them)
    result <- str_replace_all(strings, ref_pattern, "")
    
    # Trim any leading or trailing whitespace
    result <- str_trim(result)
    
    return(result)
}



_TODO:_ Use the `dplyr::mutate()` function to apply the `remove_ref` function to the `CITY` and `SYSTEM` columns

In [17]:
# sub_bike_sharing_df %>% mutate(column1=remove_ref(column1), ... )
sub_bike_sharing_df <- sub_bike_sharing_df %>%
  mutate(
    CITY = remove_ref(CITY),
    SYSTEM = remove_ref(SYSTEM)
  )

print(sub_bike_sharing_df)

# A tibble: 480 × 4
   COUNTRY   CITY                  SYSTEM           BICYCLES
   <chr>     <chr>                 <chr>            <chr>   
 1 Albania   Tirana                NA               200     
 2 Argentina Mendoza               NA               40      
 3 Argentina San Lorenzo, Santa Fe Biciudad         80      
 4 Argentina Buenos Aires          Serttel Brasil   4000    
 5 Argentina Rosario               NA               480     
 6 Australia Melbourne             PBSC & 8D        676     
 7 Australia Brisbane              3 Gen. Cyclocity 2000    
 8 Australia Melbourne             4 Gen. oBike     1250    
 9 Australia Sydney                4 Gen. oBike     1250    
10 Australia Sydney                4 Gen. Ofo       600     
# ℹ 470 more rows


_TODO:_ Use the following code to check whether all reference links are removed:


In [18]:
# result %>% 
#     select(CITY, SYSTEM, BICYCLES) %>% 
#     filter(find_reference_pattern(CITY) | find_reference_pattern(SYSTEM) | find_reference_pattern(BICYCLES))

sub_bike_sharing_df %>%
  select(CITY, SYSTEM, BICYCLES) %>%
  filter(find_reference_pattern(CITY) | find_reference_pattern(SYSTEM) | find_reference_pattern(BICYCLES))


CITY,SYSTEM,BICYCLES
<chr>,<chr>,<chr>
Brussels,3 Gen. Cyclocity,4115[22]
"Limassol (& Agios Dometios, Aglandjia, Dali, Engomi, Latsia, Pallouriotissa, Strovolos)",3 Gen. Smoove,310[59]
Prague,NA,500[72]
Prague 7,4 Gen. Ofo,[75]
Prostějov,3 Gen. nextbike,180[76]
Ostrava,3 Gen. nextbike,600[77]
Farsø,2 Gen,[78]
Batumi,3 Gen. SmooveKey,370[114]
Darmstadt,3 & 4 Gen. Call a Bike flex,350 [125]


# TASK: Extract the numeric value using regular expressions


_TODO:_ Write a custom function using `stringr::str_extract` to extract the first digital substring match and convert it into numeric type For example, extract the value '32' from `32 (including 6 rollers) [162]`.


In [40]:
# Extract the first number
# extract_num <- function(columns){
#     # Define a digital pattern
#     digitals_pattern <- "Define a pattern matching a digital substring"
#     # Find the first match using str_extract
#     # Convert the result to numeric using the as.numeric() function
# }
extract_num <- function(columns) {
  # Define a digital pattern
  digitals_pattern <- "[^0-9]"
  # Find the first match using str_extract
  str_extract(columns, digitals_pattern)
  # Convert the result to numeric using the as.numeric() function
  columns <- as.numeric(columns)
}

_TODO:_ Use the `dplyr::mutate()` function to apply `extract_num` on the `BICYCLES` column


In [43]:
# Use the mutate() function on the BICYCLES column
sub_bike_sharing_df  <- sub_bike_sharing_df  %>% mutate(BICYCLES = extract_num(BICYCLES))

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `BICYCLES = extract_num(BICYCLES)`.
Caused by warning in `extract_num()`:
! NAs introduced by coercion”


_TODO:_ Use the summary function to check the descriptive statistics of the numeric `BICYCLES` column


In [44]:
summary(sub_bike_sharing_df$BICYCLES)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
    5.0   100.0   335.5  2052.3  1468.2 78000.0     104 

_TODO:_ Write the cleaned bike-sharing systems dataset into a csv file called `bike_sharing_systems.csv`


In [ ]:
# Write dataset to `bike_sharing_systems.csv`


# References:


If you need to refresh your memory about regular expressions, please refer to this good Regular Expression cheat sheet:

<a href="https://www.rstudio.com/wp-content/uploads/2016/09/RegExCheatsheet.pdf" target="_blank">Basic Regular Expressions in R</a>


# Next Steps


Great! Now you have cleaned up the bike-sharing system dataset using regular expressions. Next, you will use other `tidyverse` functions to perform data wrangling on the bike-sharing demand dataset.


## Authors

<a href="https://www.linkedin.com/in/yan-luo-96288783/" target="_blank">Yan Luo</a>


### Other Contributors

Jeff Grossman


<!-- ## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description           |
| ----------------- | ------- | ---------- | ---------------------------- |
| 2021-04-08        | 1.0     | Yan        | Initial version created      |
|                   |         |            |                              |
|                   |         |            |                              |-->

![footer](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/SNIBMfooter-pngWUtMJhgfnxKkgB_tuc-gpg.png)
